                                                 Домашнє завдання N 7

Так як ми працюємо з рекомендованними системами, по завданню нам радять використовувати бібліотеку Surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Тому давайте імпортуємо цю бібліотеку і також завантажимо потрібний датасет movielens на 100 к даних. Думаю, нам буде достатньо цих даних для виконання домашнього завдання:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [2]:
dataset_path = './ml-100k/u.data'
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(dataset_path, reader=reader)

data

Давайте спробуємо вивести перші 5 рядків, щоб побачити, які дані містить наш датасет. Для цього використаємо трохи інший ніж зазвичай метод виводу:

In [4]:
N = 5
data_raw = data.raw_ratings

for i in range(N):
    print(data_raw[i])



('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


Окей, супер!

Тепер давайте побудуємо модель за допомогою 3 алгоритмів розграхунків: SVD, SVD++, NMF та вкористаємо cross_validation для пошуку параметрів RMSE та MAE

Почнемо з SVD:

In [5]:
algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9311  0.9363  0.9460  0.9391  0.9269  0.9359  0.0066  
MAE (testset)     0.7325  0.7365  0.7443  0.7410  0.7324  0.7374  0.0047  
Fit time          0.39    0.42    0.39    0.41    0.40    0.40    0.01    
Test time         0.04    0.07    0.04    0.07    0.04    0.05    0.02    


{'test_rmse': array([0.93112719, 0.93629776, 0.94600246, 0.93906143, 0.92688007]),
 'test_mae': array([0.73251491, 0.73654293, 0.74433736, 0.74103845, 0.73243902]),
 'fit_time': (0.389301061630249,
  0.4224691390991211,
  0.39185023307800293,
  0.4097752571105957,
  0.39676690101623535),
 'test_time': (0.03851604461669922,
  0.07335495948791504,
  0.03801584243774414,
  0.07043886184692383,
  0.03802824020385742)}

Тепер те саме зробимо з SVD++:

In [6]:
algo_svdpp = SVDpp()

cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9188  0.9187  0.9212  0.9163  0.9208  0.9191  0.0018  
MAE (testset)     0.7215  0.7221  0.7227  0.7178  0.7206  0.7209  0.0017  
Fit time          6.15    6.11    6.09    6.11    6.14    6.12    0.02    
Test time         1.39    1.27    1.29    1.25    1.25    1.29    0.05    


{'test_rmse': array([0.91875943, 0.91873051, 0.92119281, 0.91626255, 0.92080444]),
 'test_mae': array([0.72151795, 0.72210513, 0.72266949, 0.71778294, 0.7206186 ]),
 'fit_time': (6.150278806686401,
  6.112303256988525,
  6.089151859283447,
  6.1125168800354,
  6.142814874649048),
 'test_time': (1.3883302211761475,
  1.2664849758148193,
  1.2880439758300781,
  1.2465968132019043,
  1.2509219646453857)}

І також використовуючи NMF модель:

In [7]:
algo_nmf = NMF()

cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9677  0.9658  0.9608  0.9715  0.9595  0.9651  0.0044  
MAE (testset)     0.7585  0.7577  0.7578  0.7629  0.7538  0.7581  0.0029  
Fit time          0.50    0.53    0.52    0.55    0.52    0.53    0.02    
Test time         0.07    0.03    0.08    0.03    0.08    0.06    0.02    


{'test_rmse': array([0.96770482, 0.96582384, 0.96084661, 0.97145851, 0.959537  ]),
 'test_mae': array([0.75851115, 0.75772405, 0.75776103, 0.76289388, 0.75379381]),
 'fit_time': (0.5023519992828369,
  0.5264167785644531,
  0.5239381790161133,
  0.5527348518371582,
  0.523327112197876),
 'test_time': (0.07097816467285156,
  0.031109094619750977,
  0.07519292831420898,
  0.03355097770690918,
  0.07794475555419922)}

Ми отримали результати оцінок RMSE та MAE всіх 3 алгоритмів. Давайте проаналізуємо отримані дані та зробимо висновки:

Алгоритм SVD: Має середнє значення RMSE приблизно 0.9345 і MAE приблизно 0.7373. Цей алгоритм показав непогані результати з помірним часом навчання та тестування.

Алгоритм SVD++: Показав кращі результати за RMSE (середнє 0.9194) та MAE (середнє 0.7213) порівняно з SVD, що свідчить про вищу точність прогнозування. Однак, час навчання та тестування значно вищий, що є важливим аспектом для великих наборів даних або обмежених обчислювальних ресурсів.

Алгоритм NMF: Має найвищі значення помилок RMSE (середнє 0.9648) та MAE (середнє 0.7572) серед трьох розглянутих алгоритмів, що вказує на нижчу точність прогнозування порівняно з SVD та SVD++. Час навчання та тестування подібний до SVD.

Як висновок, можемо зазначити, що найбільш вдалим тут здається вибір алгоритму SVD++, так як в нас датасет не такий великий. Якби ми взяли датасет не на 100к а на 1М даних, то досить велика ймовірність, що краще б було тоді застосовувати алгоритм SVD.

Ми знайшли оптимальний алгоритм на основі пошуку параметрів RMSE,MAE через cross_validation.

Тепер же давайте застосуємо метод GridSearchCV для пошуку оптимальних гіперпараметрів. Для цього використаємо розділ документації бібліотеки surprise: Tune algorithm parameters with GridSearchCV.

Зробимо це для алгоритму SVD++, так як він показався найбільш оптимальним, хоча достатньо ресурсно-залежний:

In [8]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD++: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD++: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd_pp = SVDpp(**gs.best_params['rmse'])
algo_mae_svd_pp = SVDpp(**gs.best_params['mae'])

RMSE for SVD++: 0.963640051179698
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD++: 0.7722645522976695
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


Тепер, коли у нас є модель, яка оптимізована за обома метриками і підібрані гіперпараметри, ми можемо провести оцінку на тестовому наборі для перевірки точності. Для цього розділимо наш датасет на тренувальний і тестовий у пропорції 80 на 20:

In [9]:
trainset, testset = train_test_split(data, test_size=0.2)

Тепер навчимо нашу модель на тренувальному наборі:

In [10]:
algo_rmse_svd_pp.fit(trainset)
algo_mae_svd_pp.fit(trainset)

Тепер отримаємо прогнози на нашому тестовому наборі:

In [11]:
predictions_rmse = algo_rmse_svd_pp.test(testset)
predictions_mae = algo_mae_svd_pp.test(testset)

І тепер проведемо оцінку моделі:

In [12]:
accuracy_rmse_svd_pp = accuracy.rmse(predictions_rmse)
accuracy_mae_svd_pp = accuracy.mae(predictions_mae)

RMSE: 0.9708
MAE:  0.7765


Давайте зробимо пошук оптимальних гіперпараметрів для алгоритмів SVD та NMF і потім оцінимо отримані результати.

Для алгоритму SVD:

In [13]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd = SVD(**gs.best_params['rmse'])
algo_mae_svd = SVD(**gs.best_params['mae'])

RMSE for SVD: 0.9636867801661001
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD: 0.7722431923077995
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [14]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_svd.fit(trainset)
algo_mae_svd.fit(trainset)

predictions_rmse = algo_rmse_svd.test(testset)
predictions_mae = algo_mae_svd.test(testset)

accuracy_rmse_svd = accuracy.rmse(predictions_rmse)
accuracy_mae_svd = accuracy.mae(predictions_mae)

RMSE: 0.9626
MAE:  0.7695


Для алгоритму NMF:

In [15]:
param_grid = {"n_epochs": [5, 10], "n_factors": [15, 50, 100], "reg_pu": [0.05, 0.1, 0.15], "reg_qi": [0.05, 0.1, 0.15]}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for NMF: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for NMF: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_nmf = NMF(**gs.best_params['rmse'])
algo_mae_nmf = NMF(**gs.best_params['mae'])

RMSE for NMF: 0.9676427310583892
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.15, 'reg_qi': 0.15}
MAE for NMF: 0.7614110412359664
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.15, 'reg_qi': 0.15}


In [35]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_nmf.fit(trainset)
algo_mae_nmf.fit(trainset)

predictions_rmse = algo_rmse_nmf.test(testset)
predictions_mae = algo_mae_nmf.test(testset)

accuracy_rmse_nmf = accuracy.rmse(predictions_rmse)
accuracy_mae_nmf = accuracy.mae(predictions_mae)

RMSE: 0.9618
MAE:  0.7550


Здається, алгоритм NMF показав найкращий результат за метрикою RMSE (0.9618) та майже найкращий за MAE (0.7550), що свідчить про добру здатність моделі загалом передбачати оцінки, особливо з точки зору квадратичних помилок. Але це можливо повʼязано з правильним підбором параметрів в param_grid.

По методу cross_validation найкращим був SVD++. 
Можливо, це повʼязано з тим, що в param_grid для GridSearchCV я задав не найкращі параметри для SVD++...